# Section 4

Today:
<ul>
<li>Convolutions</li>
<li>Pooling</li>
<li>TensorBoard</li>
</ul>

<em>Some code from tensorflow.org documentation.</em>

## Extra Resources
<ul>
<li><a href='https://www.tensorflow.org/api_docs/python/tf/name_scope'>TF Name Scoping</a></li>
<li><a href='http://cs231n.github.io/convolutional-networks/'>CS 231n Notes on CNNs</a></li>
</ul>

## TF Convolutions and Pooling
<a href='https://www.tensorflow.org/tutorials/layers'>A Guide to TF Layers: Building a Convolutional Neural Network</a><br/>
<a href='https://www.tensorflow.org/versions/r1.3/api_docs/python/tf/layers/conv2d'>tf.layers.conv2d</a><br/>
<a href='https://www.tensorflow.org/api_docs/python/tf/layers/max_pooling2d'>tf.layers.max_pooling2d</a><br/>

In [1]:
# Setup and load data
import tensorflow as tf
import numpy as np

def load(filename, W=64, H=64):
    data = np.fromfile(filename, dtype=np.uint8).reshape((-1, W*H*3+1))
    images, labels = data[:, :-1].reshape((-1,H,W,3)), data[:, -1]
    return images, labels

train_image_data, train_label_data = load('tux_train.dat')
val_image_data, val_label_data = load('tux_val.dat')

In [2]:
tf.reset_default_graph()

sess = tf.Session()

# Let's build our first CNN
I = tf.placeholder(tf.float32, (None, 64, 64, 3))
white_inputs = (I - 100.) / 72.
labels = tf.placeholder(tf.int64, (None))

# Convolutional Layer #1
conv1 = tf.contrib.layers.conv2d(
  inputs=white_inputs,
  num_outputs=20,
  kernel_size=[5, 5],
  stride=2,
  padding="same",
  scope='conv1')
print(conv1)

# Pooling Layer #1
pool1 = tf.contrib.layers.max_pool2d(inputs=conv1, kernel_size=[2, 2], stride=2, scope='pool1')
print(pool1)

# Convolutional Layer #2
conv2 = tf.contrib.layers.conv2d(
    inputs=pool1,
    num_outputs=64,
    kernel_size=[5, 5],
    stride=2,
    padding="same",
    scope='conv2')
print(conv2)

# Pooling Layer #2
pool2 = tf.contrib.layers.max_pool2d(inputs=conv2, kernel_size=[2, 2], stride=2, scope='pool2')
print(pool2)

# Flatten pooled features
pool2_flat = tf.reshape(pool2, [-1, np.prod(pool2.get_shape().as_list()[1:])])
print(pool2_flat)

dense = tf.contrib.layers.fully_connected(inputs=pool2_flat, num_outputs=1024, activation_fn=tf.nn.relu)
print(dense)

# Logits Layer
logits = tf.contrib.layers.fully_connected(inputs=dense, num_outputs=10)
print(logits)

correct_prediction = tf.equal(tf.argmax(logits,1), labels)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))
print(loss)

optimizer = tf.train.AdamOptimizer(0.001, 0.9, 0.999)
print(optimizer)

train_op = optimizer.minimize(loss)
print(train_op)

Tensor("conv1/Relu:0", shape=(?, 32, 32, 20), dtype=float32)
Tensor("pool1/MaxPool:0", shape=(?, 16, 16, 20), dtype=float32)
Tensor("conv2/Relu:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("pool2/MaxPool:0", shape=(?, 4, 4, 64), dtype=float32)
Tensor("Reshape:0", shape=(?, 1024), dtype=float32)
Tensor("fully_connected/Relu:0", shape=(?, 1024), dtype=float32)
Tensor("fully_connected_1/Relu:0", shape=(?, 10), dtype=float32)
Tensor("Mean_1:0", shape=(), dtype=float32)
name: "Adam"
op: "NoOp"
input: "^Adam/update_conv1/weights/ApplyAdam"
input: "^Adam/update_conv1/biases/ApplyAdam"
input: "^Adam/update_conv2/weights/ApplyAdam"
input: "^Adam/update_conv2/biases/ApplyAdam"
input: "^Adam/update_fully_connected/weights/ApplyAdam"
input: "^Adam/update_fully_connected/biases/ApplyAdam"
input: "^Adam/update_fully_connected_1/weights/ApplyAdam"
input: "^Adam/update_fully_connected_1/biases/ApplyAdam"
input: "^Adam/Assign"
input: "^Adam/Assign_1"



## TensorBoard
<a href='https://www.tensorflow.org/get_started/summaries_and_tensorboard'>TensorBoard Introduction</a><br/>
<a href='https://www.tensorflow.org/api_guides/python/summary'>Summary variables</a><br/>
<a href='https://www.tensorflow.org/api_docs/python/tf/summary/merge_all'>merge_all</a><br/>
<a href='https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter'>FileWriter</a><br/>
<a href='https://research.googleblog.com/2017/09/build-your-own-machine-learning.html'>New TensorBoard API</a><br/><br/>
Let's use the graph we used when introducing convolutions but now add summary operations so we can see the graph and monitor training.

In [3]:
# First let's set up TensorBoard
train_writer = tf.summary.FileWriter('/tmp/section4/train', sess.graph)
validation_writer = tf.summary.FileWriter('/tmp/section4/validation')

sess.run(tf.global_variables_initializer())

In [4]:
# Add some summary operations
acc = tf.summary.scalar('Accuracy', accuracy)

# Add visualizations of our 1st layers weights
with tf.variable_scope('conv1') as scope:
    tf.get_variable_scope().reuse_variables()
    weights = tf.get_variable('weights')
    tf.summary.image('filters', tf.transpose(weights, [3, 0, 1, 2]), max_outputs=10)
    tf.summary.histogram('filters', tf.get_variable('weights'))

# Check the distribution on our outputs
tf.summary.histogram('outputs', logits)
        
# Merge them together
merged = tf.summary.merge_all()

In [5]:
# Start TensorBoard from the command line
# tensorboard --logdir=/tmp/section4

In [6]:
# Train your network and watch it on TensorBoard
BS = 32
train_count = 0
for epoch in range(20):
    np.random.seed(epoch)
    np.random.shuffle(train_image_data)
    np.random.seed(epoch)
    np.random.shuffle(train_label_data)
    # Train
    # Go through the entire dataset once
    for i in range(0, train_image_data.shape[0]-BS+1, BS):
        # Train a single batch
        batch_images, batch_labels = train_image_data[i:i+BS], train_label_data[i:i+BS]
        summary, _ = sess.run([merged, train_op], feed_dict={I: batch_images, labels: batch_labels})
        train_writer.add_summary(summary, train_count)
        train_count += 1
    validation_writer.flush()
    train_writer.flush()
    # Validation Accuracy
    summary, acc = sess.run([merged, accuracy], feed_dict={I: val_image_data, labels: val_label_data})
    validation_writer.add_summary(summary, train_count)
    print('Validation accuracy at epoch %s: %s' % (epoch, acc))
    

Validation accuracy at epoch 0: 0.644683
Validation accuracy at epoch 1: 0.65363
Validation accuracy at epoch 2: 0.776841
Validation accuracy at epoch 3: 0.83589
Validation accuracy at epoch 4: 0.846626
Validation accuracy at epoch 5: 0.832566
Validation accuracy at epoch 6: 0.830266
Validation accuracy at epoch 7: 0.840491
Validation accuracy at epoch 8: 0.83001
Validation accuracy at epoch 9: 0.848159
Validation accuracy at epoch 10: 0.836656
Validation accuracy at epoch 11: 0.854294
Validation accuracy at epoch 12: 0.856595
Validation accuracy at epoch 13: 0.861963
Validation accuracy at epoch 14: 0.865031
Validation accuracy at epoch 15: 0.851483
Validation accuracy at epoch 16: 0.856339
Validation accuracy at epoch 17: 0.858384
Validation accuracy at epoch 18: 0.855828
Validation accuracy at epoch 19: 0.871421
